# r/CasualConversations Database creation
This Jupyter Notebook is meant to create a database from submissions to r/CasualConversations. It first extracts submissions and then saves them into a special database file such that feature can be created from the submissions. This notebook is part of a project that tries to predict the flair of a submission on CasualConversations.

In [3]:
import praw # Source code : https://github.com/praw-dev/praw
            # helpful page: https://praw.readthedocs.io/en/latest/code_overview/praw_models.html

# Initialising a connection to Reddit
reddit = praw.Reddit(user_agent='Doing some stuff', client_id='hir_Veg3Rs-Svw', client_secret="giRkbKknUe4Oy4EcoGYA1N_Y0sA",
                    username='', password='')

In [4]:
# General modules and function(s)
import numpy as np
from datetime import datetime
import re
import os
import json
import ijson
from tqdm import tqdm
import pandas as pd

def utc_time(timestamp):
    '''Changes Unix to UTC time'''
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

## Filter online JSON database of reddit posts in a certain month
Certain templated flairs exist as of March 2019 in r/CasualConversation. However, in previous months, those templated flairs were different or users used their own flairs. Therefore, the approach is to find all the submissions in the JSON database that are made in r/CasualConversations and filter these submissions based on the flairs that can be recognised a topic of discussion. For example, the submissions that have a flair containing the removal reason of the post or a random flair thought up by a user (e.g. 🍕 This subreddit is now sponsored by Domino's), will be filtered out.

### Set-up

In [5]:
r_CasualConversation = reddit.subreddit('CasualConversation')

In [6]:
if 'all_submission_ids' not in locals():
    all_submissions = []
    all_submission_ids = []
    print('"all_submissions" and "all_submission_ids" have been initialised.')
else:
    print('"all_submissions" and "all_submission_ids" already exist')

"all_submissions" and "all_submission_ids" have been initialised.


### Extracting the submissions that are made in r/CasualConversation

In [7]:
all_submissions = []
all_flairs = []
submissions_found = 0

print('Submissions found :')
with open('RS_2018-10') as json_data:
    for line in json_data:
        json_line = json.loads(line)
        if(json_line['subreddit'] == 'CasualConversation'):
            all_submissions.append(json_line)
            all_flairs.append(json_line['link_flair_text'])
            submissions_found += 1
            if(json_line['link_flair_text'] != None):
                print('Count : ' + str(submissions_found) + ' '*(10-len(str(submissions_found))) +
                      'Title : ' + json_line['title'][:40] + ' '*(50-len(str(json_line['title'][:40]))) +
                      'Flair : ' + json_line['link_flair_text']
                     )
            else:
                print('Count : ' + str(submissions_found) + ' '*(10-len(str(submissions_found))) +
                      'Title : ' + json_line['title'][:40] + ' '*(50-len(str(json_line['title'][:40]))) +
                      'Flair : ' + 'None'
                     )

Submissions found :
Count : 1         Title : The worst part about being an adult is t          Flair : no context⇢
Count : 2         Title : I hate social anxiety and being alone in          Flair : banned topic⇢
Count : 3         Title : wendy from ozark is so god damn gorgeous          Flair : no context⇢
Count : 4         Title : Feeling bad about buying food at the air          Flair : None
Count : 5         Title : New job starts tomorrow!                          Flair : None
Count : 6         Title : Good Weekend after a long time!                   Flair : None
Count : 7         Title : It’s late at night and I’m alone with my          Flair : None
Count : 8         Title : Had a pretty good week yet...                     Flair : None
Count : 9         Title : The fact that ecosystems thrive on balan          Flair : no context⇢
Count : 10        Title : Are the new Macs REALLY that bad?                 Flair : None
Count : 11        Title : Need some "Movie Character Day" co

Count : 90        Title : I'm 22 and have a desire to be adopted!           Flair : None
Count : 91        Title : Which wig should I choose? (link to poll          Flair : no context⇢
Count : 92        Title : I’ve been seeing a lot of posts lately j          Flair : banned topic⇢
Count : 93        Title : I want to try scuba diving but doing som          Flair : abandoned⇢
Count : 94        Title : Anyone here? I feel like talking.                 Flair : None
Count : 95        Title : My fiancé found an abandoned kitten and           Flair : None
Count : 96        Title : How old are you, and how much money do y          Flair : no context⇢
Count : 97        Title : Any ideas for pun based house decoration          Flair : None
Count : 98        Title : Reddit followers                                  Flair : None
Count : 99        Title : Cubicle/office workers, how do you stay           Flair : None
Count : 100       Title : Is “having a charity” some type of incom          Flair

Count : 179       Title : i do not know                                     Flair : None
Count : 180       Title : I need some karma so I can post on this           Flair : no context⇢
Count : 181       Title : Laugh to start your work week: Adam Driv          Flair : no context⇢
Count : 182       Title : Today is my birthday and I didn't even n          Flair : None
Count : 183       Title : Roped my first goat today                         Flair : None
Count : 184       Title : I hate the fact that i dont have anyone           Flair : None
Count : 185       Title : I need a little bit of karma so I can ma          Flair : None
Count : 186       Title : So I have been listening to this pianist          Flair : None
Count : 187       Title : Gushers are so tasty that they've never           Flair : no context⇢
Count : 188       Title : I keep seeing this girl on campus                 Flair : banned topic⇢
Count : 189       Title : I wasn’t sure where to post this or why           Flai

Count : 267       Title : Is there any way to step back?                    Flair : None
Count : 268       Title : Can I have some encouragement/How should          Flair : None
Count : 269       Title : I hope the physical keyboard for cell ph          Flair : None
Count : 270       Title : I cannot motivate myself to do anything           Flair : None
Count : 271       Title : Little scary dream where I am a creep             Flair : None
Count : 272       Title : I could have sworn the actor for Charlie          Flair : no context⇢
Count : 273       Title : My doctor told me there is nothing I can          Flair : None
Count : 274       Title : it’s my birthday!                                 Flair : None
Count : 275       Title : I don't get why the US just doesn't lega          Flair : no context⇢
Count : 276       Title : Would it be weird to go up to a guy, tel          Flair : banned topic⇢
Count : 277       Title : Album recommendations?                            Flair : no 

Count : 356       Title : My Self-Realosation                               Flair : abandoned⇢
Count : 357       Title : Pet owners who got a pet after you reach          Flair : None
Count : 358       Title : Am I a feminist?                                  Flair : banned topic⇢
Count : 359       Title : You ever feel like everyone wants to mak          Flair : None
Count : 360       Title : I constantly think of other people if th          Flair : no context⇢
Count : 361       Title : What are your reviews of medium.com?              Flair : abandoned⇢
Count : 362       Title : Why does some people have some concerns           Flair : abandoned⇢
Count : 363       Title : I slapped a kid in the face today                 Flair : None
Count : 364       Title : I feel like tf2 should have a slot in ma          Flair : no context⇢
Count : 365       Title : I constantly think of other people if th          Flair : no context⇢
Count : 366       Title : Ok i get its odd but i love masks   

...

Count : 6894      Title : I learned how to whistle at the age of 2          Flair : None
Count : 6895      Title : I have to be awake in 2 hours                     Flair : None
Count : 6896      Title : Conflicted on whether or not to return t          Flair : banned topic⇢
Count : 6897      Title : Anybody here have any crazy dirty old ki          Flair : Music
Count : 6898      Title : What is so appealing but clubs / bars fo          Flair : None
Count : 6899      Title : What's the reason so many people In the           Flair : banned topic⇢
Count : 6900      Title : My whole body hurts :D                            Flair : no context⇢
Count : 6901      Title : How did your parents react when you told          Flair : banned topic⇢
Count : 6902      Title : Anyone else down for another r/place?             Flair : no context⇢
Count : 6903      Title : Parents of Reddit where do you dispose o          Flair : None
Count : 6904      Title : My theory on trust is you can trust anyo  

Count : 6982      Title : What drives you forward and gives you pu          Flair : None
Count : 6983      Title : Brazil just elected a new president. Let          Flair : None
Count : 6984      Title : What's your biggest fear?                         Flair : None
Count : 6985      Title : Emotions impair judgement.                        Flair : None
Count : 6986      Title : I think Nazis were actually vampires              Flair : None
Count : 6987      Title : My husband likes to call my bra my “batt          Flair : no context⇢
Count : 6988      Title : What are the funniest names you’ve ever           Flair : None
Count : 6989      Title : Any other young women out there feel wei          Flair : None
Count : 6990      Title : Is it just me or is Reddit kind of depre          Flair : None
Count : 6991      Title : Is our subconscious devoid of our issues          Flair : no context⇢
Count : 6992      Title : “Can you just buy it in the store?”               Flair : abandoned⇢
C

Count : 7070      Title : Which is worse: complete isolation or ha          Flair : None
Count : 7071      Title : I spend an inordinate amount of time on           Flair : None
Count : 7072      Title : In this day and age, why is it still nec          Flair : None
Count : 7073      Title : Happy Halloween!!! (Almost)                       Flair : None
Count : 7074      Title : A fun little game, if anyone is so inter          Flair : no context⇢
Count : 7075      Title : I‘ve been banned from r/sex should I be           Flair : None
Count : 7076      Title : Just a lonely teenager looking for a con          Flair : no context⇢
Count : 7077      Title : Just a lonely teenager looking for a con          Flair : no context⇢
Count : 7078      Title : What if condiments were deemed illegal/           Flair : None
Count : 7079      Title : Halloween Costume Ideas for a Cook?               Flair : None
Count : 7080      Title : It's 23:05 at night, I'm 32 years old an          Flair : no co

Count : 7158      Title : I made my own news and pop culture websi          Flair : banned topic⇢
Count : 7159      Title : Is there some expresions in your languag          Flair : None
Count : 7160      Title : Got sent home from work today                     Flair : banned topic⇢
Count : 7161      Title : Another typhoon is striking, the season           Flair : no context⇢
Count : 7162      Title : People never talk to me...                        Flair : banned topic⇢
Count : 7163      Title : Facebook ads on videos are far too long           Flair : no context⇢
Count : 7164      Title : If it wasn't for illegal immigrants we w          Flair : no context⇢
Count : 7165      Title : Thank you X 100                                   Flair : None
Count : 7166      Title : If your car is slowly losing coolant, fi          Flair : None
Count : 7167      Title : Painted Rocks                                     Flair : abandoned⇢
Count : 7168      Title : My childhood fantasy was to ge

Count : 7246      Title : Today I had a hard time in class at the           Flair : None
Count : 7247      Title : How many, and what kind, shoes do you gu          Flair : None
Count : 7248      Title : L.A. face with Oakland booty..                    Flair : no context⇢
Count : 7249      Title : L.A. face with I Oakland booty...                 Flair : no context⇢
Count : 7250      Title : Any of you have any No Nut November stor          Flair : no context⇢
Count : 7251      Title : My girlfriend of 2 years cancellde our h          Flair : banned topic⇢
Count : 7252      Title : Getting a massage for the first time...           Flair : None
Count : 7253      Title : Got a Halloween costume but nowhere to g          Flair : banned topic⇢
Count : 7254      Title : So, im alone on haloween...                       Flair : None
Count : 7255      Title : Does anyone else get anxiety when you we          Flair : None
Count : 7256      Title : I'm wearing a halloween costume today, d     

### Filtering based on flair

In [9]:
flair_templates = list(r_CasualConversation.flair.link_templates)

flair_ids = []
for flair_template in flair_templates:
    flair_ids.append(flair_templates)
    print(flair_template['text'])

RequestException: error with request HTTPSConnectionPool(host='www.reddit.com', port=443): Max retries exceeded with url: /api/v1/access_token (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001F9B1CBA0B8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [10]:
all_flairs_set = set(all_flairs)
flair_column = []
count_column = []

for flair in all_flairs_set:
    if(flair != None):
        flair_column.append(flair)
        count_column.append(all_flairs.count(flair))
#         print(flair + ' ' + str(all_flairs.count(flair)))
flair_count_df = pd.DataFrame(data={'flair':flair_column, 'count':count_column})
flair_count_df = flair_count_df.sort_values(by='count',ascending=False)
flair_count_df

flair  count
22                                    no context⇢   1487
11                                  banned topic⇢    965
10                                     abandoned⇢    522
9                                           Music    103
4                              Movies &amp; Shows     55
18                                         Gaming     52
12                                           Book     28
19                                     no context     16
23                                          CAPS⇢     15
20                                           Neat      9
21                                         bored⇢      8
6                        abandoned⇢ banned topic⇢      6
8                        banned topic⇢ no context      5
15                                       removed⇢      5
16                                       privacy⇢      3
3                                        respect⇢      3
14                                          r/all      2
17                                       mod post      2
5                                        Mod Post      2
7                            :topic: Weekly Topic      1
1                        :food: Food &amp; Drinks      1
13                         banned topic⇢ privacy⇢      1
2   🍕 This subreddit is now sponsored by Domino's      1
0                               :shield: Mod Post      1

### Opening and saving submissions

In [23]:
# Saving is commented out to prevent accidental saving
# with open('submissions_CasualConversation_2018-10.json', 'w+') as save_file:
#     for submission in all_submissions:
#         save_file.write("%s\n" % json.dumps(submission))

In [25]:
saved_submissions = []
saved_flairs = []

with open('submissions_CasualConversation_2018-10.json', 'r') as save_file:
    save_lines = save_file.readlines()
    for line in save_lines:
        json_line = json.loads(line)
        save_submissions.append(json_line)
        save_flairs.append(json_line['link_flair_text'])

In [29]:
save_file.close()